# Football Analysis of the Top 5 European Leagues

*This project aims to investigate the relationship between a team's total points count, total goals & total expected goals at the end of the season for Europe's top 5 leagues (2014 - 2019)*


The Definitions for all of the metrics used in the project can be found here: **GITHUB LINK**

**Luck Definition:**  Difference between league points and expected points each season

**Goal Luck:** Difference between league goals and expected goals each season

**Question: What impact (If any) does luck play in the end league position for each team?**

**To track this, we will look at the league position for each individual team by season, studying their improvements/decline in league position alongside their points and expected points**

## 1. Import Functions

In [49]:
import pandas as pd
import numpy as np
import requests

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Clean Datasets

**Datasource: https://www.kaggle.com/slehkyi/web-scraping-football-statistics-2014-now**

**Dataset:** Data from top 5 leagues in csv format which was pulled from the website **understat.com** using the web scraping methodology described in the link above

In [52]:
# Data Source using API
api_data_pull = requests.get('https://www.kaggle.com/slehkyi/extended-football-stats-for-european-leagues-xg')

db = pd.DataFrame(api_data_pull)

In [53]:
# Read csv data using Pandas 
team_data = pd.read_csv('C:/Users/costelloem/ucd-assignment/Data/stats_by_team.csv')
team_data.head()

,Unnamed: 0,Unnamed: 1,position,team,matches,wins,draws,loses,scored,missed,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,La_liga,2014,1,Barcelona,38,30,4,4,110,21,...,28.444293,7.444293,24.727907,73.049305,5.683535,16.367593,489,114,94.0813,0.0813
1,La_liga,2014,2,Real Madrid,38,30,2,6,118,38,...,42.607198,4.607198,38.890805,47.213090,10.209085,12.929510,351,153,81.7489,-10.2511
2,La_liga,2014,3,Atletico Madrid,38,23,9,6,67,29,...,29.069107,0.069107,26.839271,25.748737,8.982028,9.237091,197,123,73.1353,-4.8647
3,La_liga,2014,4,Valencia,38,22,11,5,70,32,...,39.392572,7.392572,33.446477,16.257501,8.709827,7.870225,203,172,63.7068,-13.2932
4,La_liga,2014,5,Sevilla,38,23,7,8,71,45,...,47.862742,2.862742,41.916529,20.178070,8.276148,9.477805,305,168,67.3867,-8.6133


In [8]:
# Re-Write dataset inserting column names for column 1 & 2
team_stats = team_data.rename(index=int, columns={'Unnamed: 0': 'league', 'Unnamed: 1': 'year'}) 
team_stats.head()

,league,year,position,team,matches,wins,draws,loses,scored,missed,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,La_liga,2014,1,Barcelona,38,30,4,4,110,21,...,28.444293,7.444293,24.727907,73.049305,5.683535,16.367593,489,114,94.0813,0.0813
1,La_liga,2014,2,Real Madrid,38,30,2,6,118,38,...,42.607198,4.607198,38.890805,47.213090,10.209085,12.929510,351,153,81.7489,-10.2511
2,La_liga,2014,3,Atletico Madrid,38,23,9,6,67,29,...,29.069107,0.069107,26.839271,25.748737,8.982028,9.237091,197,123,73.1353,-4.8647
3,La_liga,2014,4,Valencia,38,22,11,5,70,32,...,39.392572,7.392572,33.446477,16.257501,8.709827,7.870225,203,172,63.7068,-13.2932
4,La_liga,2014,5,Sevilla,38,23,7,8,71,45,...,47.862742,2.862742,41.916529,20.178070,8.276148,9.477805,305,168,67.3867,-8.6133


In [35]:
# Examine all columns & data types
team_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588 entries, 0 to 587
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   league        588 non-null    object 
 1   year          588 non-null    int64  
 2   position      588 non-null    int64  
 3   team          588 non-null    object 
 4   matches       588 non-null    int64  
 5   wins          588 non-null    int64  
 6   draws         588 non-null    int64  
 7   loses         588 non-null    int64  
 8   scored        588 non-null    int64  
 9   missed        588 non-null    int64  
 10  pts           588 non-null    int64  
 11  xG            588 non-null    float64
 12  xG_diff       588 non-null    float64
 13  npxG          588 non-null    float64
 14  xGA           588 non-null    float64
 15  xGA_diff      588 non-null    float64
 16  npxGA         588 non-null    float64
 17  npxGD         588 non-null    float64
 18  ppda_coef     588 non-null    

In [13]:
# Show that the dataset has no NULL values:
team_stats.isnull().sum()

league          0
year            0
position        0
team            0
matches         0
wins            0
draws           0
loses           0
scored          0
missed          0
pts             0
xG              0
xG_diff         0
npxG            0
xGA             0
xGA_diff        0
npxGA           0
npxGD           0
ppda_coef       0
oppda_coef      0
deep            0
deep_allowed    0
xpts            0
xpts_diff       0
dtype: int64

In [24]:
# What leagues are available in this dataset? We're only interested in the top 5
team_stats['league'].unique()

# Filter dataset for Europe's top 5 Leagues: PL, Serie A, Bundesliga, La Liga & Ligue 1:
team_stats = team_stats[team_stats.league != 'RFPL']

In [21]:
# List of Europe's top 5 leagues:
team_stats['league'].unique()

array(['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1'], dtype=object)

## 3. Analyse Dataset

In [43]:
# Show average team position, pts and expected pts from 2014 - 2019
team_mean_stats = team_stats.groupby(['league','team'])[['position', 'wins', 'xG', 'xG_diff', 'pts', 'xpts_diff']].mean()

team_mean_stats.head()

position       wins         xG   xG_diff  \
league     team                                                             
Bundesliga Augsburg             12.166667  10.000000  42.291191 -0.875476   
           Bayer Leverkusen      5.166667  16.333333  55.920765 -3.912569   
           Bayern Munich         1.000000  25.833333  79.799526 -8.367141   
           Borussia Dortmund     3.500000  19.000000  67.922701 -3.743966   
           Borussia M.Gladbach   5.666667  16.166667  55.164596 -0.335404   

                                      pts  xpts_diff  
league     team                                       
Bundesliga Augsburg             39.000000   1.705583  
           Bayer Leverkusen     56.333333  -0.467317  
           Bayern Munich        82.166667  -5.801550  
           Borussia Dortmund    64.666667   0.535283  
           Borussia M.Gladbach  55.500000  -2.613100

In [45]:
# Reset Index 
team_mean_stats = team_stats.groupby(['league','team'])[['position', 'wins', 'xG', 'xG_diff', 'pts', 'xpts_diff']].mean().reset_index()

team_mean_stats.head()

,league,team,position,wins,xG,xG_diff,pts,xpts_diff
0,Bundesliga,Augsburg,12.166667,10.000000,42.291191,-0.875476,39.000000,1.705583
1,Bundesliga,Bayer Leverkusen,5.166667,16.333333,55.920765,-3.912569,56.333333,-0.467317
2,Bundesliga,Bayern Munich,1.000000,25.833333,79.799526,-8.367141,82.166667,-5.801550
3,Bundesliga,Borussia Dortmund,3.500000,19.000000,67.922701,-3.743966,64.666667,0.535283
4,Bundesliga,Borussia M.Gladbach,5.666667,16.166667,55.164596,-0.335404,55.500000,-2.613100


## 4. Functions ...

## 5. Visualise

## 6. Insights